In [ ]:
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.linear_model import ElasticNet
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import random
import numpy

from numpy import *
from sklearn import *
from pandas import *
from scipy import stats

xl = pd.ExcelFile('data2.xlsx')
xl.sheet_names # we'll take 7th
dfs = {sheet: xl.parse(sheet) for sheet in xl.sheet_names}
data1 = dfs['7']
data2 = dfs['1'].loc[:,['Patient','Age at Diagnosis']].drop([554]).drop_duplicates()
# import datas/et1
data3 = pd.read_csv('data1.csv')

combined_data = data1.set_index('Patient').join(data2.set_index('Patient')).join(data3.set_index('Patient'))

combined_data['label'] = (combined_data['Patient Type'] == 'Healthy').astype(int)
combined_data = combined_data.drop(['Patient Type'],axis=1)
print('The number of samples and features are %d and %d, respectively'%(combined_data.shape[0],combined_data.shape[1]))


x = combined_data.iloc[:, 0:44]
x[isnan(x)] = 0
y=combined_data.iloc[:,44]

In [ ]:
def SFLA_ENet(x_traincv, y_traincv,x_testcv, y_testcv,alpha,l1_ratio):
    clf = ElasticNet(alpha,l1_ratio).fit(x_traincv, y_traincv)
    y_score = clf.predict(x_testcv)
    fpr, tpr, threshold = metrics.roc_curve(y_testcv, y_score, pos_label=1)
    roc_auc = metrics.auc(fpr,tpr)
    
    return roc_auc 

In [ ]:
def SFLA_ENet_CV(x_train, y_train,n,alpha,l1_ratio):
    '''
    n: number of splits for k-fold
    
    '''
    KF = KFold(n_splits=n,shuffle=True, random_state=920)
    f = []
    for train_indexcv,test_indexcv in KF.split(x_train):
        x_traincv, x_testcv = x_train.iloc[train_indexcv][:], x_train.iloc[test_indexcv][:]
        y_traincv, y_testcv = y_train.iloc[train_indexcv][:], y_train.iloc[test_indexcv][:]
        fq = SFLA_ENet(x_traincv, y_traincv,x_testcv, y_testcv,alpha,l1_ratio)
        f.append(fq) 
    f = mean(f)
    return f

In [ ]:
def SFLA(num_parameter,num_global,num_local,m,n,q,n1,rangeAlpha,rangeL1_ratio,x_train,y_train):
    '''
    num_parameter: int, number of parameter to optimize
    
    num_global: int, the maximum number of global iterations
    
    num_local: int, the maximum number of local iterations
    
    m : int, the number of memeplexes
    
    n : int, the number of frogs in each memeplex
    
    q : int, the number of frogs in submemeplex
    
    n1:  number of splits for cross validation for inner loop
    
    rangeAlpha: list, float, range of parameter Alpha,eg.[10**-2, 10]
    
    rangeL1_ratio: list, float, range of parameter L1_ratio,eg.[0, 1]

    x_train: feature

    y_train: lable

    '''

    #--- Step 0--Initialize parameters ---#
    sizeAlpha = 2
    sizeL1_ratio = 2
    max_step =  [(rangeAlpha[1]-rangeAlpha[0])/sizeAlpha,(rangeL1_ratio[1]-rangeL1_ratio[0])/sizeL1_ratio]# maximum step size
    
    #--- Step 1--Generate initial population ---#
    frogAlpha = random.uniform(rangeAlpha[0],rangeAlpha[1],m*n)
    frogL1_ratio = random.uniform(rangeL1_ratio[0],rangeL1_ratio[1],m*n)
    frog = c_[frogAlpha,frogL1_ratio]

    # Compute the performance value for each frog on validation data #
    KF = KFold(n_splits=n1,shuffle=True, random_state=920)
    f = zeros((m*n,n1))
    j = 0
    for train_indexcv,test_indexcv in KF.split(x_train):
        x_traincv, x_testcv = x_train.iloc[train_indexcv][:], x_train.iloc[test_indexcv][:]
        y_traincv, y_testcv = y_train.iloc[train_indexcv][:], y_train.iloc[test_indexcv][:]
        for i in range(m*n):
            f[i,j] = SFLA_ENet(x_traincv, y_traincv,x_testcv, y_testcv,frogAlpha[i],frogL1_ratio[i])
        j+=1
    f = f.mean(axis=1)
    f_parameter = c_[f,frog]

    #--- Step 2--Rank frogs ---#
    f_parameter = f_parameter[argsort(f_parameter[:,0])[::-1]]


    #######--- Global search start---######
    i_global = 0
    flag = 0
    fBest_iteration = f_parameter[0,0]
    weights = [2*(n+1-j)/(n*(n+1)) for j in range(1,n+1)] # weights of ranked frogs in each memeplex
    while i_global < num_global:
        frog_gb = f_parameter[0,0] # mark the global best frog      
        #--- Step 3--Partition frogs into memeplexes ---#
        memeplexes = zeros((m,n,num_parameter+1)) # [memeplexes, frog in memeplex,[f,C,Gamma] ]
        for i in range(m):
            memeplexes[i] = f_parameter[linspace(i,m*n+i,num=n,endpoint=False,dtype=int)]

        #######--- Local search start---######
        #--- Step 4--Memetic evolution within each memeplex ---#
        im = 0 # the number of memeplexes that have been optimized
        while im < m:
            i_local = 0 # counts the number of local evolutionary steps in each memeplex
            while i_local < num_local:

                #--- Construct a submemeplex ---#
                rValue = random.random(n)*weights # random value with probability weights
                subindex = sort(argsort(rValue)[::-1][0:q]) # index of selected frogs in memeplex 
                submemeplex = memeplexes[im][subindex] # form submemeplex

                #--- Improve the worst frog's position ---#
                # Learn from local best Pb #
                Pb = submemeplex[0] # mark the best frog in submemeplex
                Pw = submemeplex[q-1] # mark the worst frog in memeplex
                S = (Pb-Pw)[1:]*(Pb-Pw)[0] 
                Uq = Pw[1:]+S
                # Check feasible space and the performance #
                if (rangeAlpha[0] <= Uq[0] <=rangeAlpha[1]) and (rangeL1_ratio[0] <= Uq[1] <=rangeL1_ratio[1]): # check feasible space
                    fq = SFLA_ENet_CV(x_train, y_train,n1,Uq[0],Uq[1])
                    if fq < Pw[0]: # if no improvement of performance,learn from global best randomly #
                        S = random.random(num_parameter)*(frog_gb-Pw)[1:]
                        for i in range(num_parameter):
                            if S[i] > 0:
                                S[i] = min(S[i],max_step[i])
                            else:
                                S[i] = min(S[i],-max_step[i])
                        Uq = Pw[1:]+S
                        if (rangeAlpha[0] <= Uq[0] <=rangeAlpha[1]) and (rangeL1_ratio[0] <= Uq[1] <=rangeL1_ratio[1]): # check feasible space
                            fq = SFLA_ENet_CV(x_train, y_train,n1,Uq[0],Uq[1])
                            if fq < Pw[0]: # if no improvement of performance, randomly generate a new frog
                                Uq = [random.uniform(rangeAlpha[0],rangeAlpha[1]),random.uniform(rangeL1_ratio[0],rangeL1_ratio[1])]
                                fq = SFLA_ENet_CV(x_train, y_train,n1,Uq[0],Uq[1])
                        else: # if not in the feasible space, randomly generate a new frog
                            Uq = [random.uniform(rangeAlpha[0],rangeAlpha[1]),random.uniform(rangeL1_ratio[0],rangeL1_ratio[1])]
                            fq = SFLA_ENet_CV(x_train, y_train,n1,Uq[0],Uq[1])           
                else: # if not in the feasible space, learn from global best randomly 
                    S = random.random(num_parameter)*(frog_gb-Pw)[1:]
                    for i in range(num_parameter):
                        if S[i] > 0:
                            S[i] = min(S[i],max_step[i])
                        else:
                            S[i] = min(S[i],-max_step[i])
                    Uq = Pw[1:]+S
                    if (rangeAlpha[0] <= Uq[0] <=rangeAlpha[1]) and (rangeL1_ratio[0] <= Uq[1] <=rangeL1_ratio[1]): # check feasible space
                        fq = SFLA_ENet_CV(x_train, y_train,n1,Uq[0],Uq[1])
                        if fq < Pw[0]: # if no improvement of performance, randomly generate a new frog
                            Uq = [random.uniform(rangeAlpha[0],rangeAlpha[1]),random.uniform(rangeL1_ratio[0],rangeL1_ratio[1])]
                            fq = SFLA_ENet_CV(x_train, y_train,n1,Uq[0],Uq[1])
                    else: # if not in the feasible space, randomly generate a new frog
                        Uq = [random.uniform(rangeAlpha[0],rangeAlpha[1]),random.uniform(rangeL1_ratio[0],rangeL1_ratio[1])]
                        fq = SFLA_ENet_CV(x_train, y_train,n1,Uq[0],Uq[1])

                #--- Upgrade the memeplex ---# 
                memeplexes[im][subindex[q-1]] = r_[fq,Uq]
                memeplexes[im] =  memeplexes[im][argsort( memeplexes[im][:,0])[::-1]]            

                i_local += 1

            im += 1
        #######--- Local search end---######    

        #--- Step 5--Shuffle memeplexes ---#
        f_parameter =  memeplexes.reshape(m*n,num_parameter+1)
        f_parameter = f_parameter[argsort(f_parameter[:,0])[::-1]]


        i_global += 1

        #--- Step 6--Check convergence ---#
        if f_parameter[0,0] > 0.9999:
            print('The program was terminated because it reached the optimization goal with f = %.3f' %f_parameter[0,0])
            break
            
        fBest_iteration = r_[fBest_iteration,f_parameter[0,0]] 

    #######--- Global search end---######
        
    return (f_parameter[0],fBest_iteration)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import time
start = time.process_time()
n_repeat = 10 # number of time that nested k-fold cross validation repeat
n_outer = 10 # number of splits for outer loop
n_inner = 5 # number of splits for inner loop
rangeAlpha = [10**-2, 2] # list, float, range of parameter C,eg.[10**-2, 10**2]
rangeL1_ratio = [0, 1] # list, float, range of parameter Gamma,eg.[10**-6, 1]

num_parameter = 2# number of parameter to optimize
num_global = 30# the maximum number of global iterations
num_local = 20# the maximum number of local iterations
m =4 # the number of memeplexes
n = 8 # the number of frogs in each memeplex
q = 5 # the number of frogs in submemeplex

auc_split = []
for i in range(n_repeat):
    ##---Classification with nested 10*10-fold cross-validation---##
    #--- x is feature, y is lable, n is number of fold
    #---  define K-fold cross validation ---#
    KF = StratifiedKFold(n_outer,shuffle=True, random_state=920+i)
    
    for train_index,test_index in KF.split(x,y):
        #---  Seperate traing set and test set ---#
        x_train, x_test = x.iloc[train_index][:], x.iloc[test_index][:]
        y_train = y.iloc[train_index][:]
        #---  normalize feature values to zero mean and unit variance ---#
        # this makes comparing weights more meaningful
        #   feature value 0 means the average value for that features
        #   feature value of +1 means one standard deviation above average
        #   feature value of -1 means one standard deviation below average
        scaler = preprocessing.StandardScaler()  
        x_train = scaler.fit_transform(x_train)  
        x_test  = scaler.transform(x_test)
        #---Unbalanced class---#
        from imblearn.over_sampling import SMOTE
        # define SMOTE
        smo = SMOTE(random_state=42)
        x_train, y_train = smo.fit_sample(x_train,y_train)
        x_train =  pd.DataFrame(x_train)
        x_test = pd.DataFrame(x_test)
        y_train = pd.DataFrame(y_train)
        #---  Fill NaN age ---#
        x_train[isnan(x_train)] = 0
        x_test[isnan(x_test)] = 0
        ##---  optimize SVM with SFLA---##
        f_parameter,fBest_iteration = SFLA(num_parameter,num_global,num_local,m,n,q,n_inner,rangeAlpha,rangeL1_ratio,x_train,y_train)
        ##---  creat and train the model ---##
        clf = ElasticNet(alpha=f_parameter[1],l1_ratio=f_parameter[2])
        clf.fit(x_train, y_train)
        
        # Calculate AUC
        fpr, tpr, threshold = roc_curve(y.iloc[test_index][:].values, clf.predict(x_test), pos_label=1)
        auc_split.append(auc(fpr,tpr))
        print(auc_split)

end = time.process_time()
print('AElastic-Net takes '+str(end - start)+'seconds.\n') 

print('AUC in each split:',auc_split)
AUC = mean(auc_split)
print('Mean of AUC: %.4f' % (AUC))

# Confidence interval
lower = percentile(auc_split, 2.5)
upper = percentile(auc_split, 97.5)
print('95%% Confidence interval: [%.4f, %.4f]' % (lower, upper))